Буду работать с данными https://www.kaggle.com/datasets/lokeshparab/amazon-products-dataset/
моя основная задача понять как работает ранжирование и написать простенькую програму локального поискавика по данным

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
!unzip "/content/archive.zip" -d "/content/data"

Archive:  /content/archive.zip
  inflating: /content/data/Air Conditioners.csv  
  inflating: /content/data/All Appliances.csv  
  inflating: /content/data/All Books.csv  
  inflating: /content/data/All Car and Motorbike Products.csv  
  inflating: /content/data/All Electronics.csv  
  inflating: /content/data/All English.csv  
  inflating: /content/data/All Exercise and Fitness.csv  
  inflating: /content/data/All Grocery and Gourmet Foods.csv  
  inflating: /content/data/All Hindi.csv  
  inflating: /content/data/All Home and Kitchen.csv  
  inflating: /content/data/All Movies and TV Shows.csv  
  inflating: /content/data/All Music.csv  
  inflating: /content/data/All Pet Supplies.csv  
  inflating: /content/data/All Sports Fitness and Outdoors.csv  
  inflating: /content/data/All Video Games.csv  
  inflating: /content/data/Amazon Fashion.csv  
  inflating: /content/data/Amazon Pharmacy.csv  
  inflating: /content/data/Amazon-Products.csv  
  inflating: /content/data/Baby Bath Skin 

In [2]:
folder_path = '/content/data'
df = pd.DataFrame()

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        full_path = os.path.join(folder_path, file)
        data = pd.read_csv(full_path)
        df = pd.concat([df, data], ignore_index=True)

In [3]:
df['discount_price'] = (
    df['discount_price']
    .str.replace('₹', '', regex=False)
    .str.replace(',', '', regex=False)
    .replace(['', 'NaN', 'null', 'None'], np.nan)
    .astype(float)
    .fillna(0)
)

In [4]:
df['actual_price'] = (
    df['actual_price']
    .str.replace('₹', '', regex=False)
    .str.replace(',', '', regex=False)
    .replace(['', 'NaN', 'null', 'None'], np.nan)
    .astype(float)
    .fillna(0)
)

In [5]:
df['ratings'] = (
    df['ratings']
    .str.replace('₹', '', regex=False)
    .str.replace(',', '', regex=False)
    .replace(['', 'NaN', 'null', 'None', 'Get', 'FREE'], np.nan)
    .astype(float)
    .fillna(0)
)

In [7]:
# df['no_of_ratings'] = (
#     df['no_of_ratings']
#     .str.replace('₹', '', regex=False)
#     .str.replace(',', '', regex=False)
#     .replace(['', 'NaN', 'null', 'None', 'Get', 'FREE', 'FREE Delivery by Amazon', 'Only 1 left in stock.', 'Only 2 left in stock.', 'Usually dispatched in 2 to 3 days.', 'Usually dispatched in 3 to 5 days.', 'Usually dispatched in 4 to 5 days.', 'Usually dispatched in 5 to 6 days.', 'Usually dispatched in 6 to 7 days.', 'Usually dispatched in 7 to 8 days.', 'Usually dispatched in 3 to 4 weeks.'], np.nan)
#     .astype(float)
#     .fillna(0)
# )

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103170 entries, 0 to 1103169
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   name            1103170 non-null  object 
 1   main_category   1103170 non-null  object 
 2   sub_category    1103170 non-null  object 
 3   image           1103170 non-null  object 
 4   link            1103170 non-null  object 
 5   ratings         1103170 non-null  float64
 6   no_of_ratings   751582 non-null   object 
 7   discount_price  1103170 non-null  float64
 8   actual_price    1103170 non-null  float64
 9   Unnamed: 0      551585 non-null   float64
dtypes: float64(4), object(6)
memory usage: 84.2+ MB


In [7]:
df.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0
0,Pigeon by Stovekraft Amaze Plus Electric Kettl...,appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Pigeon-Amaze-Plus-1-5-Lt...,3.9,"128,941",599.0,1245.0,NaN
1,Pigeon Polypropylene Mini Handy and Compact Ch...,appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Pigeon-Stovekraft-Plasti...,4.1,"274,505",199.0,545.0,NaN
2,Glun Multipurpose Portable Electronic Digital ...,appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Multipurpose-Portable-El...,3.8,365,199.0,899.0,NaN
3,beatXP Kitchen Scale Multipurpose Portable Ele...,appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/beatXP-Multipurpose-Port...,3.7,"3,290",299.0,1999.0,NaN
4,Bajaj DX-6 1000W Dry Iron with Advance Solepla...,appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Bajaj-Majesty-1000-Watt-...,4.2,"24,380",625.0,1400.0,NaN


In [8]:
from IPython.display import Image, display
from random import randint

for i in range(3):
  index = randint(0, len(df))
  url = df.iloc[index]['image']
  display(Image(url=url, width=300))

Сначала поработаю с текстовыми признаками с помощью bm25

---


$$
BM25(q, d) = \sum_{i=1}^{n} IDF(q_i) \times \frac{TF(q_i, d) \times (k_1 + 1)}{TF(q_i, d) + k_1 \times \left(1 - b + b \times \frac{|d|}{avgdl}\right)}
$$

$TF(q_i, d)$ - частота слова $q_i$ в документе

$avgdl$ - средняя длина документов

$$
IDF(q_i) = \log \frac{N - n(q_i) + 0.5}{n(q_i) + 0.5}
$$

$n(q_i)$ - количество документов содержащих $q_i$.

$N$ - общее число документов.

---


bm25 это доработанная версия TF-IDF с помощью дополнительных коэфициентов мы отдаем больший приоритет коротким названиям и штрафуем за множественное повторение одного слова

In [11]:
!pip install rank_bm25

In [6]:
import numpy as np
import string

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = text.split()
    return tokens

df['tokenized_name'] = df['name'].apply(preprocess_text)
df['tokenized_main_category'] = df['main_category'].apply(preprocess_text)
df['tokenized_sub_category'] = df['sub_category'].apply(preprocess_text)

display(df.head())

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0,tokenized_name,tokenized_main_category,tokenized_sub_category
0,Pigeon by Stovekraft Amaze Plus Electric Kettl...,appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Pigeon-Amaze-Plus-1-5-Lt...,3.9,"128,941",599.0,1245.0,NaN,"[pigeon, by, stovekraft, amaze, plus, electric...",[appliances],"[all, appliances]"
1,Pigeon Polypropylene Mini Handy and Compact Ch...,appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Pigeon-Stovekraft-Plasti...,4.1,"274,505",199.0,545.0,NaN,"[pigeon, polypropylene, mini, handy, and, comp...",[appliances],"[all, appliances]"
2,Glun Multipurpose Portable Electronic Digital ...,appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Multipurpose-Portable-El...,3.8,365,199.0,899.0,NaN,"[glun, multipurpose, portable, electronic, dig...",[appliances],"[all, appliances]"
3,beatXP Kitchen Scale Multipurpose Portable Ele...,appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/beatXP-Multipurpose-Port...,3.7,"3,290",299.0,1999.0,NaN,"[beatxp, kitchen, scale, multipurpose, portabl...",[appliances],"[all, appliances]"
4,Bajaj DX-6 1000W Dry Iron with Advance Solepla...,appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Bajaj-Majesty-1000-Watt-...,4.2,"24,380",625.0,1400.0,NaN,"[bajaj, dx6, 1000w, dry, iron, with, advance, ...",[appliances],"[all, appliances]"


In [7]:
from rank_bm25 import BM25Okapi

In [26]:
from rank_bm25 import BM25Okapi

tokenized_products = [t_p for t_p in df['tokenized_name']]
tokenized_main_category = [t_p for t_p in df['tokenized_main_category']]
tokenized_sub_category = [t_p for t_p in df['tokenized_sub_category']]

bm25_name = BM25Okapi(tokenized_products)
bm25_main_category = BM25Okapi(tokenized_main_category)
bm25_sub_category = BM25Okapi(tokenized_sub_category)

In [ ]:
bm25_name = 0
bm25_main_category = 0
bm25_sub_category = 0

In [16]:
queries = [
    "watch",
    "glue",
    "batterfly"
]

In [17]:
for query in queries:
    tokenized_query = preprocess_text(query)
    scores = 0.4 * bm25_name.get_scores(tokenized_query) + 0.3 * bm25_main_category.get_scores(tokenized_query) + 0.3 * bm25_sub_category.get_scores(tokenized_query)
    print(f"\nЗапрос: '{query}'")
    for (product, score, index) in sorted(zip(df['name'], scores, range(len(df))), key=lambda x: x[1], reverse=True)[:1]:
        print(f"  {product} -> BM25: {score:.3f}")
        url = df.iloc[index]['image']
        display(Image(url=url, width=300))


Запрос: 'watch'
  FARP Digital Watch led Watch Side Purple Colour Womens Watch Ladies Watch Girls Watch Boys Watch Mens Watch -> BM25: 2.566



Запрос: 'glue'
  SOLDTRUE Huda Beauty Eye-Lash Glue False Eyelash Adhesive Glue Eyelashes Extension Glue 1pc -> BM25: 5.419



Запрос: 'batterfly'
  NiyatiFab Purple Color Dual Batterfly with Black Color Small Heart Combo Watch - for Girls -> BM25: 4.631


Сделаю синтетические данные для предобучения модели

In [23]:
query = "smart watch"

def assign_relevance(row, score_name, score_main_cat, score_sub_cat):
    text_score = (
        0.4 * score_name[row.name] +
        0.3 * score_main_cat[row.name] +
        0.3 * score_sub_cat[row.name]
    )

    quality_score = 0.1 * (row['ratings'] / 5.0)

    total_score = text_score + quality_score

    if total_score >= 0.7:
        return 2  # Полностью релевантен
    elif total_score >= 0.3:
        return 1  # Частично релевантен
    else:
        return 0  # Не релевантен

tokenized_query = preprocess_text(query)
score_name = bm25_name.get_scores(tokenized_query)
score_main_cat = bm25_main_category.get_scores(tokenized_query)
score_sub_cat = bm25_sub_category.get_scores(tokenized_query)
df['relevance'] = df.apply(lambda row: assign_relevance(row, score_name, score_main_cat, score_sub_cat), axis=1)

df.sort_values('relevance', ascending=False).head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0,tokenized_name,tokenized_main_category,tokenized_sub_category,relevance
497273,GROOT Rich Analog Black Color Dial Men Watch -...,accessories,Watches,https://m.media-amazon.com/images/I/71I5WlewLl...,https://www.amazon.in/GROOT-Analog-Black-Color...,0.0,NaN,619.0,1021.0,NaN,"[groot, rich, analog, black, color, dial, men,...",[accessories],[watches],2
497274,GROOT Style Analog Blue and Black Color Dial M...,accessories,Watches,https://m.media-amazon.com/images/I/71xkPH3oJ-...,https://www.amazon.in/GROOT-Style-Analog-Black...,0.0,NaN,795.0,1311.0,NaN,"[groot, style, analog, blue, and, black, color...",[accessories],[watches],2
70962,OTTOMATE Sense Connect 5 Star Rated Smart BLDC...,appliances,Kitchen & Home Appliances,https://m.media-amazon.com/images/I/6107sWo50R...,https://www.amazon.in/OTTOMATE-Sense-Connect-T...,3.6,794,4170.0,6300.0,NaN,"[ottomate, sense, connect, 5, star, rated, sma...",[appliances],"[kitchen, home, appliances]",2
497275,GROOT Italian Designer Analog Blue and Gold Co...,accessories,Watches,https://m.media-amazon.com/images/I/71Y5NNXtJq...,https://www.amazon.in/GROOT-Italian-Designer-A...,0.0,NaN,649.0,1070.0,NaN,"[groot, italian, designer, analog, blue, and, ...",[accessories],[watches],2
497276,TIMENTER Tread Analog White and Gold Color Dia...,accessories,Watches,https://m.media-amazon.com/images/I/71F9UxaiDy...,https://www.amazon.in/TIMENTER-Tread-Analog-Wh...,0.0,NaN,714.0,1178.0,NaN,"[timenter, tread, analog, white, and, gold, co...",[accessories],[watches],2


Сгенерировал 100 синтетических запросов

In [8]:
 all_search_queries = [
    # Electronics (25)
    "wireless headphones", "bluetooth earbuds", "smartphone android", "apple iphone",
    "smart watch", "fitness tracker", "bluetooth speaker", "portable power bank",
    "usb-c charger", "tablet case", "gaming mouse", "mechanical keyboard", "4k monitor",
    "laptop for work", "drawing tablet", "e-book reader", "smart watch with ECG",
    "wireless charger", "bluetooth headset", "streaming webcam", "recording microphone",
    "portable SSD", "wi-fi router", "smart speaker", "VR headset",

    # Home & Kitchen (20)
    "coffee maker", "air fryer", "instant pot", "kitchen blender", "stand mixer",
    "food processor", "toaster oven", "microwave oven", "vacuum cleaner", "air purifier",
    "humidifier", "essential oil diffuser", "bed sheet set", "memory foam pillow",
    "weighted blanket", "cookware set", "kitchen knives", "non-stick pan", "baking sheets",
    "food storage containers",

    # Fashion & Clothing (15)
    "running shoes", "sneakers for men", "women's dress", "jeans for women", "winter jacket",
    "rain coat", "athletic shorts", "yoga pants", "business shirt", "casual t-shirt",
    "wool socks", "swimwear", "handbag for women", "backpack for laptop", "sunglasses polarized",

    # Books & Media (15)
    "science fiction books", "romance novels", "business books", "cookbook for beginners",
    "children's story books", "self-help books", "programming textbooks", "fantasy book series",
    "mystery thriller", "audio books subscription", "kindle unlimited", "coloring books for adults",
    "educational books for kids", "biography of entrepreneurs", "graphic novels",

    # Toys & Games (10)
    "lego sets for kids", "board games for family", "educational toys", "video games ps5",
    "nintendo switch games", "outdoor play equipment", "puzzle games", "action figures",
    "stem learning toys", "baby toys",

    # Sports & Outdoors (10)
    "yoga mat", "dumbbell set", "exercise bike", "camping tent", "hiking backpack",
    "fishing rod", "bicycle for adults", "tennis racket", "basketball shoes", "water bottle insulated",

    # Beauty & Personal Care (5)
    "skincare set", "hair dryer", "electric toothbrush", "perfume for women", "makeup brush set"
]

Создам простенький класификатор вход 4 float (скоры по имени и категориям и рейтинг)

In [9]:
from torch import nn

model = nn.Sequential(
    nn.Linear(4, 10),
    nn.ReLU(),
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 3)
)

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
import numpy as np
from rank_bm25 import BM25Okapi
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [43]:
def assign_relevance_batch(scores_name, scores_main_cat, scores_sub_cat, ratings):
    text_scores = 0.4 * scores_name + 0.3 * scores_main_cat + 0.2 * scores_sub_cat
    quality_scores = 0.1 * (ratings / 100.0)
    total_scores = text_scores + quality_scores

    conditions = [
        total_scores >= 0.7,
        (total_scores >= 0.3) & (total_scores < 0.7),
        total_scores < 0.3
    ]
    choices = [2, 1, 0]  # 2=Полностью релевантен, 1=Частично, 0=Не релевантен

    return np.select(conditions, choices)

In [89]:
def prepare_data(df, queries, n_best=1000):
    all_features = []
    all_labels = []
    tokenized_names = [t_p for t_p in df['tokenized_name']]
    tokenized_main_cat = [t_p for t_p in df['tokenized_main_category']]
    tokenized_sub_cat = [t_p for t_p in df['tokenized_sub_category']]

    bm25_name = BM25Okapi(tokenized_names)
    bm25_main_category = BM25Okapi(tokenized_main_cat)
    bm25_sub_category = BM25Okapi(tokenized_sub_cat)

    for i, query in enumerate(queries):
        print(f"Обработка запроса {i+1}/{len(queries)}")
        tokenized_query = preprocess_text(query)

        scores_name = bm25_name.get_scores(tokenized_query)
        scores_main_cat = bm25_main_category.get_scores(tokenized_query)
        scores_sub_cat = bm25_sub_category.get_scores(tokenized_query)
        ratings = df['ratings'].values

        features = np.column_stack([scores_name, scores_main_cat, scores_sub_cat, ratings])

        labels = assign_relevance_batch(scores_name, scores_main_cat, scores_sub_cat, ratings)

        top_indices = np.argsort(labels)[::-1][:n_best]
        features_top = features[top_indices]
        labels_top = labels[top_indices]

        all_features.append(features_top)
        all_labels.append(labels_top)

    X = np.vstack(all_features)
    y = np.hstack(all_labels)
    return X, y

In [90]:
def create_dataloaders(X, y, batch_size=256, train_ratio=0.9, val_ratio=0.15):
    X_tensor = torch.FloatTensor(X)
    y_tensor = torch.LongTensor(y)

    dataset = TensorDataset(X_tensor, y_tensor)

    total_size = len(dataset)
    train_size = int(train_ratio * total_size)
    val_size = int(val_ratio * total_size)
    test_size = total_size - train_size - val_size

    train_dataset, val_dataset, test_dataset = random_split(
        dataset, [train_size, val_size, test_size]
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

In [14]:
def train_model(model, train_loader, val_loader, num_epochs=100, learning_rate=1e-4, mode="cpu"):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    if mode == "cuda":
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        criterion.to(device)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for batch_X, batch_y in train_loader:
            if mode == "cuda":
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += batch_y.size(0)
            train_correct += (predicted == batch_y).sum().item()

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                if mode == "cuda":
                    batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                val_total += batch_y.size(0)
                val_correct += (predicted == batch_y).sum().item()

        train_loss_avg = train_loss / len(train_loader)
        val_loss_avg = val_loss / len(val_loader)
        train_accuracy = train_correct / train_total
        val_accuracy = val_correct / val_total

        train_losses.append(train_loss_avg)
        val_losses.append(val_loss_avg)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}]')
            print(f'Train Loss: {train_loss_avg:.4f}, Train Acc: {train_accuracy:.2f}%')
            print(f'Val Loss: {val_loss_avg:.4f}, Val Acc: {val_accuracy:.2f}%')
            print('---')

    return {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accuracies': train_accuracies,
        'val_accuracies': val_accuracies
    }

In [77]:
X, y = 0, 0
train_loader, val_loader, test_loader = 0, 0, 0

In [91]:
X, y = prepare_data(df, all_search_queries, n_best=100000)

train_loader, val_loader, test_loader = create_dataloaders(X, y)
print(X[:5], y[:5])
X, y = 0, 0

Обработка запроса 1/100
Обработка запроса 2/100
Обработка запроса 3/100
Обработка запроса 4/100
Обработка запроса 5/100
Обработка запроса 6/100
Обработка запроса 7/100
Обработка запроса 8/100
Обработка запроса 9/100
Обработка запроса 10/100
Обработка запроса 11/100
Обработка запроса 12/100
Обработка запроса 13/100
Обработка запроса 14/100
Обработка запроса 15/100
Обработка запроса 16/100
Обработка запроса 17/100
Обработка запроса 18/100
Обработка запроса 19/100
Обработка запроса 20/100
Обработка запроса 21/100
Обработка запроса 22/100
Обработка запроса 23/100
Обработка запроса 24/100
Обработка запроса 25/100
Обработка запроса 26/100
Обработка запроса 27/100
Обработка запроса 28/100
Обработка запроса 29/100
Обработка запроса 30/100
Обработка запроса 31/100
Обработка запроса 32/100
Обработка запроса 33/100
Обработка запроса 34/100
Обработка запроса 35/100
Обработка запроса 36/100
Обработка запроса 37/100
Обработка запроса 38/100
Обработка запроса 39/100
Обработка запроса 40/100
Обработка

In [92]:
history = train_model(model, train_loader, val_loader, num_epochs=10, mode="cuda")

Epoch [10/10]
Train Loss: 0.0004, Train Acc: 1.00%
Val Loss: 0.0003, Val Acc: 1.00%
---


In [93]:
history

{'train_losses': [0.12437720333996796,
  0.0011118816622910854,
  0.0008162637722625684,
  0.0006651159873929702,
  0.000559503517262799,
  0.0004830656537088042,
  0.00043691227089944623,
  0.00039901459978949774,
  0.0003765017519398343,
  0.0003545337637973985],
 'val_losses': [0.0016333619760936565,
  0.0010402707178196384,
  0.0008677373609285953,
  0.0007320987648237606,
  0.0005291746107349823,
  0.00045965065506374975,
  0.00039788024532541935,
  0.0004067316000755035,
  0.0003504200814109449,
  0.00032736361105640524],
 'train_accuracies': [0.9826488888888889,
  0.9996202222222222,
  0.9997481111111111,
  0.9997998888888889,
  0.9998166666666667,
  0.9998305555555556,
  0.9998442222222222,
  0.9998671111111112,
  0.999886,
  0.9998947777777778],
 'val_accuracies': [0.999712,
  0.999675,
  0.999826,
  0.999853,
  0.999827,
  0.999826,
  0.999829,
  0.999827,
  0.999898,
  0.999829]}

In [94]:
def predict_relevance(model, query, mode="cpu", n_best=1000):
    tokenized_query = preprocess_text(query)

    scores_name = bm25_name.get_scores(tokenized_query)
    scores_main_cat = bm25_main_category.get_scores(tokenized_query)
    scores_sub_cat = bm25_sub_category.get_scores(tokenized_query)
    ratings = df['ratings'].values

    features = np.column_stack([scores_name, scores_main_cat, scores_sub_cat, ratings])
    labels = assign_relevance_batch(scores_name, scores_main_cat, scores_sub_cat, ratings)
    top_indices = np.argsort(labels)[::-1][:n_best]
    features_top1000 = features[top_indices]
    result_df = df.iloc[top_indices].copy()

    model.eval()
    with torch.no_grad():
        features_tensor = torch.FloatTensor(features_top1000).to(mode)
        outputs = model(features_tensor).cpu()
        probabilities = torch.softmax(outputs, dim=1)
        _, predictions = torch.max(outputs, 1)

    # result_df = df.copy()
    result_df['predicted_class'] = predictions.numpy()
    result_df['predicted_prob_class_0'] = probabilities.numpy()[:, 0]
    result_df['predicted_prob_class_1'] = probabilities.numpy()[:, 1]
    result_df['predicted_prob_class_2'] = probabilities.numpy()[:, 2]

    result_df['ranking_score'] = result_df['predicted_prob_class_2'] + result_df['predicted_prob_class_1'] - result_df['predicted_prob_class_0']

    return result_df.sort_values('ranking_score', ascending=False)

In [82]:
queries = ["wireless headphones", "bluetooth earbuds", "smartphone android"]

In [95]:
for query in queries:
    tokenized_query = preprocess_text(query)

    print(f"\nЗапрос: '{query}'")
    display(predict_relevance(model, query, "cuda").head())


Запрос: 'wireless headphones'


,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0,tokenized_name,tokenized_main_category,tokenized_sub_category,predicted_class,predicted_prob_class_0,predicted_prob_class_1,predicted_prob_class_2,ranking_score
38464,Willen Wireless Home Security Camera cc tv Cam...,"tv, audio & cameras",Security Cameras,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Willen-Wireless-Security...,0.0,NaN,1599.0,1999.0,NaN,"[willen, wireless, home, security, camera, cc,...","[tv, audio, cameras]","[security, cameras]",2,0.0,7.602499e-22,1.0,1.0
186560,HIKVISION 2MP Dome with inbuilt Mic DS-2CE76D0...,"tv, audio & cameras",Cameras,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/HIKVISION-inbuilt-DS-2CE...,3.4,14,1649.0,2400.0,NaN,"[hikvision, 2mp, dome, with, inbuilt, mic, ds2...","[tv, audio, cameras]",[cameras],2,0.0,7.514961e-29,1.0,1.0
186555,Osaka 18 Inches LED Ring Light 65W With Adjust...,"tv, audio & cameras",Cameras,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Osaka-Adjustable-Tempera...,4.4,7,3400.0,7999.0,NaN,"[osaka, 18, inches, led, ring, light, 65w, wit...","[tv, audio, cameras]",[cameras],2,0.0,2.396963e-23,1.0,1.0
186547,TECHNOVIEW Wireless WiFi IP CCTV Security 1080...,"tv, audio & cameras",Cameras,https://m.media-amazon.com/images/I/513QYrGfAG...,https://www.amazon.in/Technoview-Wireless-Wate...,3.2,378,2650.0,6999.0,NaN,"[technoview, wireless, wifi, ip, cctv, securit...","[tv, audio, cameras]",[cameras],2,0.0,1.472626e-22,1.0,1.0
37700,Alokit CCTV Camera V380 Pro Wireless HD Securi...,"tv, audio & cameras",Security Cameras,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Alokit-Camera-Wireless-S...,0.0,NaN,2049.0,2999.0,NaN,"[alokit, cctv, camera, v380, pro, wireless, hd...","[tv, audio, cameras]","[security, cameras]",2,0.0,9.778029e-19,1.0,1.0



Запрос: 'bluetooth earbuds'


,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0,tokenized_name,tokenized_main_category,tokenized_sub_category,predicted_class,predicted_prob_class_0,predicted_prob_class_1,predicted_prob_class_2,ranking_score
736767,"KZ ZS10 Pro in Ear Monitor Earphone,5 Driver H...","tv, audio & cameras",Headphones,https://m.media-amazon.com/images/I/51YhywqgaI...,https://www.amazon.in/ZS10-Pro-Comfortable-Iso...,4.3,99,4699.00,6999.0,1346.0,"[kz, zs10, pro, in, ear, monitor, earphone5, d...","[tv, audio, cameras]",[headphones],2,0.0,1.024821e-36,1.0,1.0
942838,AVWOO Mini Nut Bluetooth Speaker with Small an...,"tv, audio & cameras",Speakers,https://m.media-amazon.com/images/I/51Mn2J84NF...,https://www.amazon.in/Bluetooth-Speaker-elegan...,4.5,"2,212",3953.65,4186.0,1862.0,"[avwoo, mini, nut, bluetooth, speaker, with, s...","[tv, audio, cameras]",[speakers],2,0.0,5.308459e-27,1.0,1.0
521233,Portronics Resound 5W Bluetooth 5.3 Wireless S...,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/619pEIvyrG...,https://www.amazon.in/Portronics-Bluetooth-Wir...,3.9,110,899.00,1499.0,7328.0,"[portronics, resound, 5w, bluetooth, 53, wirel...","[tv, audio, cameras]","[all, electronics]",2,0.0,1.169974e-24,1.0,1.0
736885,M19 Earbud TWS Wireless In Ear Earbuds with To...,"tv, audio & cameras",Headphones,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/M19-Wireless-Earbuds-Blu...,2.6,4,640.00,1999.0,1464.0,"[m19, earbud, tws, wireless, in, ear, earbuds,...","[tv, audio, cameras]",[headphones],2,0.0,0.000000e+00,1.0,1.0
942840,TG Collections TWS Bluetooth Speaker with TV M...,"tv, audio & cameras",Speakers,https://m.media-amazon.com/images/I/41OnqDW5p6...,https://www.amazon.in/Collections-Bluetooth-Sp...,0.0,NaN,449.00,699.0,1864.0,"[tg, collections, tws, bluetooth, speaker, wit...","[tv, audio, cameras]",[speakers],2,0.0,2.501792e-23,1.0,1.0



Запрос: 'smartphone android'


,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0,tokenized_name,tokenized_main_category,tokenized_sub_category,predicted_class,predicted_prob_class_0,predicted_prob_class_1,predicted_prob_class_2,ranking_score
1000870,MI Xiaomi 138.8 cm (55 inches) 4K Ultra HD Sma...,"tv, audio & cameras",Televisions,https://m.media-amazon.com/images/I/81onGhenkU...,https://www.amazon.in/Xiaomi-inches-Android-Vi...,4.3,446,99999.0,199999.0,299.0,"[mi, xiaomi, 1388, cm, 55, inches, 4k, ultra, ...","[tv, audio, cameras]",[televisions],2,0.0,7.847271e-44,1.0,1.0
1001211,INVANTER 2022 Model Smart TV 55 INCH Frameless...,"tv, audio & cameras",Televisions,https://m.media-amazon.com/images/I/41Pp1PrN6D...,https://www.amazon.in/INVANTER-Model-Frameless...,0.0,NaN,59990.0,60990.0,640.0,"[invanter, 2022, model, smart, tv, 55, inch, f...","[tv, audio, cameras]",[televisions],2,0.0,3.042995e-38,1.0,1.0
1001212,INVANTER 2022 Model Smart TV 43 INCH Frameless...,"tv, audio & cameras",Televisions,https://m.media-amazon.com/images/I/41Pp1PrN6D...,https://www.amazon.in/INVANTER-Model-Frameless...,0.0,NaN,22490.0,36990.0,641.0,"[invanter, 2022, model, smart, tv, 43, inch, f...","[tv, audio, cameras]",[televisions],2,0.0,3.042995e-38,1.0,1.0
1001213,INVANTER 2022 Model Smart TV 43 INCH (102cm) F...,"tv, audio & cameras",Televisions,https://m.media-amazon.com/images/I/41Pp1PrN6D...,https://www.amazon.in/INVANTER-Model-Smart-Fra...,0.0,NaN,35990.0,36990.0,642.0,"[invanter, 2022, model, smart, tv, 43, inch, 1...","[tv, audio, cameras]",[televisions],2,0.0,1.077683e-39,1.0,1.0
1001214,INVANTER 2022 Model Smart TV 32 INCH (102cm) F...,"tv, audio & cameras",Televisions,https://m.media-amazon.com/images/I/41Pp1PrN6D...,https://www.amazon.in/INVANTER-Model-Smart-Fra...,0.0,NaN,13490.0,21990.0,643.0,"[invanter, 2022, model, smart, tv, 32, inch, 1...","[tv, audio, cameras]",[televisions],2,0.0,1.077683e-39,1.0,1.0


получил более менее удовлетворительные результаты но теперь попробую обучить регресию отому что мне не нравится что у меня в самом вверху все бъются в максимум

In [88]:
from torch import nn

model_regres = nn.Sequential(
    nn.Linear(4, 10),
    nn.ReLU(),
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 1)
)

In [ ]:
def assign_relevance_batch(scores_name, scores_main_cat, scores_sub_cat, ratings):
    text_scores = 0.4 * scores_name + 0.3 * scores_main_cat + 0.2 * scores_sub_cat
    quality_scores = 0.1 * (ratings / 100.0)
    total_scores = text_scores + quality_scores

    conditions = [
        total_scores >= 0.7,
        (total_scores >= 0.3) & (total_scores < 0.7),
        total_scores < 0.3
    ]
    choices = [2, 1, 0]  # 2=Полностью релевантен, 1=Частично, 0=Не релевантен

    return np.select(conditions, choices)

In [ ]:
X, y = prepare_data(df, all_search_queries)

train_loader, val_loader, test_loader = create_dataloaders(X, y)
print(X[:5], y[:5])
X, y = 0, 0